In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import nltk
import spacy
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import chi2
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectKBest
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import cross_val_score, validation_curve

C:\Users\siddh\AppData\Local\Continuum\anaconda3\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
X_train = []
X_test = []
Y_train = []
Y_test = []
X_valid = []
Y_valid = []
male = []
female = []

In [3]:
with open(r'data/training_set.txt') as training:
    for line in training:
        line = line.strip()
        lnum, chr_id, movie_id, chr_name, chr_gender, line_text, credit_list = line.split("+++$+++")
        chr_gender = chr_gender.strip()
#         ss = sid.polarity_scores(line_text)
#         sentiment = ss['compound']
        if(chr_gender.lower() == "m"):
            X_train.append(line_text)
            male.append(line_text)
            Y_train.append(0)
        elif(chr_gender.lower() == "f"):
            X_train.append(line_text)
            female.append(line_text)
            Y_train.append(1)


with open(r'data/test_set.txt') as test:
    for line in test:
        line = line.strip()
        lnum, chr_id, movie_id, chr_name, chr_gender, line_text, credit_list = line.split("+++$+++")
        chr_gender = chr_gender.strip()
#         ss = sid.polarity_scores(line_text)
#         sentiment = ss['compound']
        if(chr_gender.lower() == "m"):
            X_test.append(line_text)
            male.append(line_text)
            Y_test.append(0)
        elif(chr_gender.lower() == "f"):
            X_test.append(line_text)
            female.append(line_text)
            Y_test.append(1)




with open(r'data/validation_set.txt') as valid:
    for line in valid:
        line = line.strip()
        lnum, chr_id, movie_id, chr_name, chr_gender, line_text, credit_list = line.split("+++$+++")
        chr_gender = chr_gender.strip()
#         ss = sid.polarity_scores(line_text)
#         sentiment = ss['compound']
        if(chr_gender.lower() == "m"):
            X_valid.append(line_text)
            male.append(line_text)
            Y_valid.append(0)
        elif(chr_gender.lower() == "f"):
            X_valid.append(line_text)
            female.append(line_text)
            Y_valid.append(1)


In [4]:
X = X_train + X_test + X_valid
Y = Y_train + Y_test + Y_valid

In [5]:
indices = []
for i in range(len(X)):
  if(len(X[i]) > 0):
    answer = True 
  else:
    indices.append(i)

for index in sorted(indices, reverse=True):
    del X[index]
    del Y[index]

In [6]:
from imblearn.over_sampling import RandomOverSampler
oversample = RandomOverSampler(sampling_strategy='minority')
X, Y = oversample.fit_resample(np.array(X).reshape(-1,1), Y)

In [7]:
a = []
for phrase in X:
    a.append(phrase[0].strip())

In [8]:
X = a
print(len(X))
print(type(X[0]))

341536
<class 'str'>


In [9]:
df = pd.DataFrame()
df['text'] = X
df['target'] = Y